In [ ]:
%matplotlib inline
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, execute, Aer, IBMQ, QuantumRegister, ClassicalRegister
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *

# Loading your IBM Q account(s)
provider = IBMQ.load_account()

In [ ]:
from qiskit.extensions import *
import numpy as np
import cmath as cm
import math
from qiskit.circuit.add_control import add_control
from qiskit.circuit.library import MCXGate, ZGate
import matplotlib.pyplot as plt

In [ ]:
train_data = {
        "0000":"000000",
        "0001":"000110",
        "0010":"001001",
        "0011":"001111",
        "0100":"000100",
        "0101":"000010",
        "0110":"001101",
        "0111":"001011",
        "1000":"100000",
        "1001":"100110",
        "1010":"101001",
        "1011":"101111",
        "1100":"100100",
        "1101":"100010",
        "1110":"101101",
        "1111":"101011"
       }

n_test_features = 1
test_data = "100110"
test_feature = len(test_data)
train_feature = int(test_feature/n_test_features)
n_id = 4
features = []
for key in train_data:
    features.append(train_data[key])

In [ ]:
def create_circuit():
    psi_q = QuantumRegister(n_id,"psi_q")
    psi_ff = QuantumRegister(train_feature,"psi_ff")
    psi_fu = QuantumRegister(test_feature,"psi_fu")
    psi_k = QuantumRegister(2,"psi_k")
    c = ClassicalRegister(1, "c")
    circuit = QuantumCircuit(psi_q, psi_ff, psi_fu, psi_k,c)
    return circuit

In [ ]:
# Initialization
def initialize_circuit(circuit, test_feature):
    [psi_q, psi_ff, psi_fu, psi_k] = circuit.qregs
    circuit.h(psi_q)
    circuit.barrier()
    circuit.cx(psi_q[3],psi_ff[5])
    circuit.cx(psi_q[0],psi_ff[1])
    circuit.cx(psi_q[0],psi_ff[2])
    circuit.cx(psi_q[1],psi_ff[0])
    circuit.cx(psi_q[1],psi_ff[3])
    circuit.cx(psi_q[2],psi_ff[2])
    circuit.h(psi_k[0])
    circuit.barrier()
    for i in range(test_feature):
        if test_data[i] == '1':
            circuit.x(psi_fu[test_feature-i-1])
            
    return circuit

In [ ]:
def createP1(index):
    x = [[cm.exp(-1j*(math.pi/(2*(index+1)))),0],[0,1]]
    return x

In [ ]:
# Hamming Distance Calculation
def hamming_distance(circuit):
    [psi_q, psi_ff, psi_fu, psi_k] = circuit.qregs
    circuit.barrier()
    for j in range(n_test_features):
        for i in range(train_feature):
            circuit.cx(psi_fu[(j*train_feature)+i],psi_ff[train_feature-i-1])
    for i in range(train_feature):
        P1_mat = createP1(i)
        P1 = UnitaryGate(P1_mat,"P2")
        P2_ctrl = add_control(P1,1,label='p2', ctrl_state=0)
        circuit.append(P2_ctrl, [psi_k[0], psi_ff[i]])
    circuit.barrier()
    for i in range(train_feature):
        P1_mat = createP1(i)
        P1 = UnitaryGate(P1_mat,"P1")
        circuit.append(P1,[psi_ff[i]])
    circuit.h(psi_k[0])
    return circuit

In [ ]:
def uncompute(circuit):
    [psi_q, psi_ff, psi_fu, psi_k] = circuit.qregs
    circuit.barrier()
    circuit.h(psi_k[0])
    for i in range(train_feature):
        P1_mat = createP1(i)
        P1 = UnitaryGate(P1_mat,"P1")
        circuit.append(P1,[psi_ff[i]])
    circuit.barrier()
    for i in range(train_feature):
        P1_mat = createP1(i)
        P1 = UnitaryGate(P1_mat,"P2")
        P2_ctrl = add_control(P1,1,label='p2', ctrl_state=0)
        circuit.append(P2_ctrl, [psi_k[0], psi_ff[i]])
    circuit.barrier()
    for j in range(n_test_features):
        for i in range(train_feature):
            circuit.cx(psi_fu[(j*train_feature)+i],psi_ff[train_feature-i-1])        
    return circuit

In [ ]:
def grovers_amplification(circuit, train_feature, test_data):
    [psi_q, psi_ff, psi_fu, psi_k] = circuit.qregs
    circuit.barrier()
    circuit.h(psi_ff)
    circuit.x(psi_k[1])
    circuit.h(psi_k[1])

    for i in range(test_feature):
        if test_data[i] == '1':
            circuit.x(psi_ff[test_feature-i-1])
    mcx = MCXGate(num_ctrl_qubits=train_feature,label="mcx")
    circuit.mcx(psi_ff,psi_k[1])
    for i in range(test_feature):
        if test_data[i] == '1':
            circuit.x(psi_ff[test_feature-i-1])
    circuit.barrier()
    circuit.h(psi_ff)
    circuit.x(psi_ff)
    mcz = ZGate().control(num_ctrl_qubits=train_feature-1,label='mcz')
    circuit.append(mcz,psi_ff)
    circuit.x(psi_ff)
    circuit.h(psi_ff)
    circuit.barrier()
    circuit.h(psi_k[1])
    return circuit

In [ ]:
# Checking for c = 0 or c = 1
backend = Aer.get_backend('qasm_simulator')
shots = 1

circuit = create_circuit()
circuit = initialize_circuit(circuit, test_feature)

while True:
    circuit = hamming_distance(circuit)
    [_,_,_,psi_k] = circuit.qregs
    [c] = circuit.cregs
    circuit.measure(psi_k[0],c)
    results = execute(circuit, backend=backend, shots=shots).result()
    count = results.get_counts()
    [(count_key,_)] = count.items()
    print(count_key)
    if count_key == '0':
        break
    else:
        circuit = uncompute(circuit)

In [ ]:
ampl_features = [9]
for element in ampl_features:
    circuit = grovers_amplification(circuit, train_feature, features[element])
[psi_q, psi_ff, psi_fu, psi_k] = circuit.qregs
A = ClassicalRegister(train_feature,"a")
m_circ = QuantumCircuit(A)
main_circ = circuit + m_circ
main_circ.barrier()
for i in range(train_feature):
    main_circ.measure(psi_ff[i],A[i])
circuit.draw()

In [ ]:
backend = Aer.get_backend('qasm_simulator')
shots = 1024
results = execute(main_circ, backend=backend, shots=shots).result()
count = results.get_counts()

In [ ]:
def classical_distance(a,b):
    n = len(a)
    freq = 0
    for i in range(n):
        if a[i] != b[i]:
            freq += 1
    return freq

In [ ]:
distance1 = []

for i in range(n_test_features):
    for key in train_data:
        distance1.append(train_data[key])

if test_data in distance1:
    print(test_data," present in the database")
else:
    print(test_data," not present in the database")

sorted_values = sorted(count.values())
#print("Predicted closest neighbor: ",y_pred)

In [ ]:
y = {}
for key in count:
    key2 = key[:-2]
    y[key2] = count[key]

tmp_dict = {}

for k in y.keys():
    if k in train_data.values():
        tmp_dict[k] = y[k]
print(tmp_dict)    

In [ ]:
plt.bar(range(len(tmp_dict)), list(tmp_dict.values()), align='center')
plt.xticks(range(len(tmp_dict)), list(tmp_dict.keys()))
plt.xticks(rotation = 45)
plt.show()

In [ ]:
print(count)